In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [2]:
from pyspark.sql.functions import col, sum

- Los datos adjuntos a esta lección forman parte de la base de datos [NeurIPS 2020] Data Science for COVID-19 (DS4C) disponible en Kaggle. Estos datos hacen referencia a los casos de contagio de covid-19 en Corea del Sur

- El archivo csv Case contiene los casos reportados y el archivo csv PatientInfo contiene la información de los pacientes.

- A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

In [3]:
df_case = spark.read.csv('./data/data/Case.csv', header=True)

In [4]:
df_case.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|     Yongsan-gu| TRUE|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|      Gwanak-gu| TRUE|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|        Guro-gu| TRUE| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|   Yangcheon-gu| TRUE|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|      Dobong-gu| TRUE|     Day Care Center|       43|37.679422|127.044374|
| 1000006|   Seoul|        Guro-gu| TRUE|Manmin Central Ch...|       41|37.481059|126.894343|
| 1000007|   Seoul|from other city| TRUE|SMR Newly Planted...|       36|        -|         -|
| 1000008|   Seoul|  Dongdaemun-gu| TRUE|       Dongan Churc

In [5]:
df_case.printSchema()

root
 |--  case_id: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [6]:
df_case.withColumnRenamed(' case_id', 'case_id')

DataFrame[case_id: string, province: string, city: string, group: string, infection_case: string, confirmed: string, latitude: string, longitude: string]

In [7]:
df_case = df_case.drop(col('case_id'), col('providence'), col('group'), col('infection_case'), col('latitude'), col('longitude'))

In [8]:
df_case.show()

+--------+--------+---------------+---------+
| case_id|province|           city|confirmed|
+--------+--------+---------------+---------+
| 1000001|   Seoul|     Yongsan-gu|      139|
| 1000002|   Seoul|      Gwanak-gu|      119|
| 1000003|   Seoul|        Guro-gu|       95|
| 1000004|   Seoul|   Yangcheon-gu|       43|
| 1000005|   Seoul|      Dobong-gu|       43|
| 1000006|   Seoul|        Guro-gu|       41|
| 1000007|   Seoul|from other city|       36|
| 1000008|   Seoul|  Dongdaemun-gu|       17|
| 1000009|   Seoul|from other city|       25|
| 1000010|   Seoul|      Gwanak-gu|       30|
| 1000011|   Seoul|   Eunpyeong-gu|       14|
| 1000012|   Seoul|   Seongdong-gu|       13|
| 1000013|   Seoul|      Jongno-gu|       10|
| 1000014|   Seoul|     Gangnam-gu|        7|
| 1000015|   Seoul|        Jung-gu|        7|
| 1000016|   Seoul|   Seodaemun-gu|        5|
| 1000017|   Seoul|      Jongno-gu|        7|
| 1000018|   Seoul|     Gangnam-gu|        6|
| 1000019|   Seoul|from other city

In [9]:
df_case.count()

174

In [10]:
df_case.dropna().count()

174

In [11]:
df_case = df_case.groupBy('city').agg(sum('confirmed').alias('confirmed'))

In [12]:
df_case.show()

+---------------+---------+
|           city|confirmed|
+---------------+---------+
|     Gangnam-gu|     18.0|
|     Cheonan-si|    103.0|
|from other city|   1217.0|
|      Anyang-si|     39.0|
|      Gwanak-gu|    149.0|
|     Yongsan-gu|    139.0|
|        Dong-gu|     44.0|
|         Sejong|     39.0|
|     Gangseo-gu|      0.0|
|       Wonju-si|      4.0|
|     Suyeong-gu|      5.0|
|   Geochang-gun|     18.0|
|  Dongdaemun-gu|     17.0|
|     Dongnae-gu|     39.0|
|         Jin-gu|      4.0|
|     Yangsan-si|      3.0|
|    Changwon-si|      7.0|
|         Nam-gu|   4511.0|
|   Gyeongsan-si|     99.0|
|      Jongno-gu|     17.0|
+---------------+---------+
only showing top 20 rows



In [13]:
df_case = df_case.filter((col('city') != '-') & (col('city') != 'from other city'))

In [14]:
df_case = df_case.sort(col('confirmed').desc()).limit(3)

In [15]:
df_case.show(3)

+------------+---------+
|        city|confirmed|
+------------+---------+
|      Nam-gu|   4511.0|
|Dalseong-gun|    297.0|
|      Seo-gu|    151.0|
+------------+---------+



- Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

In [16]:
df_patient = spark.read.csv('./data/data/PatientInfo.csv', header=True)

In [17]:
df_patient.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

In [18]:
df_patient = df_patient.dropDuplicates(['patient_id'])

- ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.

In [19]:
df_patient.dropna(subset = ['infected_by']).count()

1346

- A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.

In [20]:
df_patient = df_patient.dropna(subset = ['infected_by'])

In [21]:
df_patient = df_patient.drop('released_date', 'deceased_date')

In [22]:
df_patient_female = df_patient.filter(col('sex') == 'female')

In [23]:
df_patient_female.show()

+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|patient_id|   sex|age|country|province|         city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|   state|
+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|1000000005|female|20s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000002|             2|              null|    2020-01-31|released|
|1000000006|female|50s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000003|            43|              null|    2020-01-31|released|
|1000000010|female|60s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000003|             6|              null|    2020-02-05|released|
|1000000014|female|60s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000013|            27|        2020-02-06|   

- Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [24]:
df_patient_female = df_patient_female.repartition(2)

In [26]:
# df_patient_female.write.partitionBy('province').mode('overwrite').parquet('./data/data/6_practice.parquet')